# Missing Data Summary - Synchronized Plots

Quick analysis of missing data in metal price series.

In [4]:
import pandas as pd
import numpy as np
import os

# Define data directory
data_dir = '../../data'

## Data Coverage by Metal

In [5]:
metals = ['cobalt', 'copper', 'lithium', 'nickel']
summary_data = []

for metal in metals:
    file_path = os.path.join(data_dir, f'ALL_{metal}_prices_outer.csv')
    df = pd.read_csv(file_path)
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Filter out weekends (Saturday=5, Sunday=6)
    df = df[df['Date'].dt.dayofweek < 5]
    
    # Get basic info
    total_rows = len(df)
    date_range = f"{df['Date'].min().strftime('%Y-%m-%d')} to {df['Date'].max().strftime('%Y-%m-%d')}"
    
    # Get series columns (exclude Date)
    series_cols = [col for col in df.columns if col != 'Date']
    
    summary_data.append({
        'Metal': metal.capitalize(),
        'Total Rows': total_rows,
        'Series Count': len(series_cols),
        'Date Range': date_range
    })

summary_df = pd.DataFrame(summary_data)
summary_df

,Metal,Total Rows,Series Count,Date Range
0,Cobalt,3537,6,2012-03-02 to 2025-11-18
1,Copper,2340,6,2017-01-03 to 2025-12-31
2,Lithium,2170,7,2017-05-10 to 2025-12-09
3,Nickel,2507,6,2016-04-04 to 2025-11-18


## Missing Data by Series

In [6]:
def analyze_metal_missing(metal):
    file_path = os.path.join(data_dir, f'ALL_{metal}_prices_outer.csv')
    df = pd.read_csv(file_path)
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Filter out weekends
    df = df[df['Date'].dt.dayofweek < 5]
    
    # Add year column
    df['Year'] = df['Date'].dt.year
    
    # Exclude LISAME series
    series_cols = [col for col in df.columns if col not in ['Date', 'Year'] and col != 'LISAME']
    
    # Overall stats
    overall_stats = []
    for col in series_cols:
        valid_data = df[df[col].notna()]
        if len(valid_data) > 0:
            start = valid_data['Date'].min().strftime('%Y-%m-%d')
            end = valid_data['Date'].max().strftime('%Y-%m-%d')
            date_range = f"{start} to {end}"
            
            # Calculate missing % within series date range
            series_df = df[(df['Date'] >= valid_data['Date'].min()) & 
                          (df['Date'] <= valid_data['Date'].max())]
            total = len(series_df)
            missing = series_df[col].isna().sum()
            pct_missing = (missing / total) * 100 if total > 0 else 0
        else:
            date_range = "No data"
            pct_missing = 100
        
        overall_stats.append({
            'Series': col,
            'Missing %': f"{pct_missing:.1f}",
            'Data Range': date_range
        })
    
    overall_df = pd.DataFrame(overall_stats)
    
    # Year-by-year breakdown
    yearly_stats = []
    for col in series_cols:
        # Get the valid date range for this series
        valid_data = df[df[col].notna()]
        if len(valid_data) == 0:
            continue
            
        series_start = valid_data['Date'].min()
        series_end = valid_data['Date'].max()
        
        for year in sorted(df['Year'].unique()):
            # Only include years that overlap with the series' valid range
            year_start = pd.Timestamp(f'{year}-01-01')
            year_end = pd.Timestamp(f'{year}-12-31')
            
            # Check if this year overlaps with series range
            if year_end < series_start or year_start > series_end:
                continue
            
            # Get data only within the intersection of year and series range
            effective_start = max(year_start, series_start)
            effective_end = min(year_end, series_end)
            
            year_df = df[(df['Year'] == year) & 
                        (df['Date'] >= effective_start) & 
                        (df['Date'] <= effective_end)]
            
            total = len(year_df)
            if total == 0:
                continue
                
            missing = year_df[col].isna().sum()
            pct_missing = (missing / total) * 100
            
            yearly_stats.append({
                'Series': col,
                'Year': year,
                'Missing %': f"{pct_missing:.1f}",
                'Valid Days': total - missing,
                'Total Days': total
            })
    
    yearly_df = pd.DataFrame(yearly_stats)
    
    return overall_df, yearly_df

# Show for each metal
for metal in metals:
    print(f"\n{'='*60}")
    print(f"{metal.upper()} - Overall")
    print('='*60)
    overall, yearly = analyze_metal_missing(metal)
    display(overall)
    
    print(f"\n{metal.upper()} - By Year")
    print('-'*60)
    display(yearly)


COBALT - Overall


,Series,Missing %,Data Range
0,CODALY,1.6,2017-01-02 to 2025-10-31
1,COLMEX,2.0,2012-03-02 to 2025-11-18
2,COLMEA,3.0,2017-01-03 to 2025-10-31
3,COWUXI,6.1,2017-01-03 to 2025-07-21
4,COCOMX,3.6,2020-12-28 to 2025-10-31
5,COSMMS,6.9,2023-05-04 to 2025-10-31



COBALT - By Year
------------------------------------------------------------


,Series,Year,Missing %,Valid Days,Total Days
0,CODALY,2017,0.0,260,260
1,CODALY,2018,0.0,261,261
2,CODALY,2019,0.4,260,261
3,CODALY,2020,0.8,260,262
4,CODALY,2021,0.4,260,261
5,CODALY,2022,0.0,260,260
6,CODALY,2023,3.8,250,260
7,CODALY,2024,5.0,249,262
8,CODALY,2025,5.1,206,217
9,COLMEX,2012,0.0,209,209



COPPER - Overall


...

,Series,Year,Missing %,Valid Days,Total Days
0,LIDALY,2017,0.0,160,160
1,LIDALY,2018,0.8,240,242
2,LIDALY,2019,0.4,241,242
3,LIDALY,2020,0.4,250,251
4,LIDALY,2021,1.1,258,261
5,LIDALY,2022,0.8,258,260
6,LIDALY,2023,0.8,258,260
7,LIDALY,2024,13.7,226,262
8,LIDALY,2025,8.8,198,217
9,LICOMX,2021,2.9,170,175



NICKEL - Overall


,Series,Missing %,Data Range
0,NIDALY,1.0,2017-01-02 to 2025-10-31
1,NILMEX,3.1,2016-04-04 to 2025-11-18
2,NIETFN,3.8,2023-03-22 to 2025-10-31
3,NISHFE,7.0,2017-01-03 to 2025-10-31
4,NIWUXI,4.9,2017-01-03 to 2025-07-21
5,NIINDA,1.7,2017-01-02 to 2025-10-31



NICKEL - By Year
------------------------------------------------------------


,Series,Year,Missing %,Valid Days,Total Days
0,NIDALY,2017,0.0,260,260
1,NIDALY,2018,0.0,261,261
2,NIDALY,2019,0.4,260,261
3,NIDALY,2020,0.8,260,262
4,NIDALY,2021,0.4,260,261
5,NIDALY,2022,2.3,254,260
6,NIDALY,2023,1.9,255,260
7,NIDALY,2024,3.4,253,262
8,NIDALY,2025,0.0,218,218
9,NILMEX,2016,0.0,190,190


## Key Findings

**Weekends Excluded**: Missing data percentages calculated only for weekdays (trading days)

**LISAME Excluded**: Removed from analysis due to data quality issues

**Year Boundaries**: Missing % calculated only within each series' actual data range (no penalization for years before start or after end)

**Missing Data Indicators**: Red vertical bars at subplot bottom (size 75/thickness 1)

**Main Issue**: Non-overlapping series availability - different exchanges have different start/end dates

**Recent Data**: Multiple series end Oct-Nov 2025 (only select exchanges continue)

**Plots Location**: `outputs/synchronized_plots/`